# Project Sommelier Sciencers

## Goal: 

* Find the key drivers of wine quality.
* Using 3 clustering techniques to construct 4 machine learning models to predict wine quality from the wine quality (red and white) csv's from the Data World dataset.
* Display results using 5 vizzes.

---

## <div class="alert alert-block alert-info"> Imports<div>

In [ ]:
# # My Modules
# import wrangle as wr
# import explore as ex
# import stats_conclude as sc
# import evaluate as ev

# Imports
import os

# Numbers
import pandas as pd 
import numpy as np
import scipy.stats as stats

# Vizzes
import matplotlib.pyplot as plt
import seaborn as sns

# Splitting
from sklearn.model_selection import train_test_split

# Engineering
# GET THESE IMPORTS

# Modeling
# GET THESE IMPORTS

import warnings
warnings.filterwarnings("ignore")

## <div class="alert alert-block alert-info"> Wrangle
      
### Acquire
* Data acquired from Data.World website
* Combined red and white CSV files
* Each row represents a wine
* Each column represents a feature
  
### Prepare
    
* Look at the data:
    * nulls
    * value counts
    * data types
    * numerical/categorical columns
    * names of columns
        * related columns
    
### Data Dictionary
* Explanation of dataset features
 

| Feature | Definition (measurement)|
|:--------|:-----------|
|Fixed Acidity| The fixed amount of tartaric acid. (g/L)|
|Volatile Acidity| A wine's acetic acid; (High Volatility = High Vinegar-like smell). (g/L)|
|Citric Acid| The amount of citric acid; (Raises acidity, Lowers shelf-life). (g/L)|
|Residual Sugar| Leftover sugars after fermentation. (g/L)|
|Chlorides| Increases sodium levels; (Affects color, clarity, flavor, aroma). (g/L)|
|Free Sulfur Dioxide| Related to pH. Determines how much SO2 is available. (Increases shelf-life, decreases palatability). (mg/L)|
|Total Sulfur Dioxide| Summation of free and bound SO2. (Limited to 350ppm: 0-150, low-processed, 150+ highly processed). (mg/L)|
|Density| Between 1.08 and 1.09. (Insight into fermentation process of yeast growth). (g/L)|
|pH| 2.5: more acidic - 4.5: less acidic (range)|
|Sulphates| Added to stop fermentation (Preservative) (g/L)|
|Alcohol| Related to Residual Sugars. By-product of fermentation process (vol%)|
|Quality| Score assigned between 0 and 10; 0=low, 10=best|

In [1]:
# Acquire


In [2]:
# Prepare: cleaning data


In [3]:
# Prepare: SPLIT


<div class="alert alert-block alert-info"> 
    
## Explore
    
### Questions To Answer:
1. Do chlorides affect quality?
2. Is free sulfur dioxide related to pH?
3. Is alcohol content related to residual sugars?
4. Does alcohol content affect quality of wine?


## <div class="alert alert-block alert-info">  1. Do chlorides affect quality?

$H_0$: 

$H_a$: 

In [4]:
# visualize


In [5]:
# analyze


In [7]:
# summarize


## <div class="alert alert-block alert-info">  2. Is free sulfur dioxide related to pH?

$H_0$: 

$H_a$: 

In [4]:
# visualize


In [5]:
# analyze


In [7]:
# summarize


## <div class="alert alert-block alert-info">  3. Is alcohol content related to residual sugars?

$H_0$: 

$H_a$: 

In [4]:
# visualize


In [5]:
# analyze


In [7]:
# summarize


## <div class="alert alert-block alert-info">  4. Does alcohol content affect quality of wine?

$H_0$: 

$H_a$: 

In [4]:
# visualize


In [5]:
# analyze


In [7]:
# summarize


<div class="alert alert-block alert-info">  

### Explore Takeaways:

**Target = Quality of Wine**


    

#### Features to Send In:
    
* 
* 
* 
* 
    
#### Features to Exclude:
* 
    
---
    

<div> 

<div class="alert alert-block alert-info"> 

## Preprocessing

* Encode any categoricals

* Engineered Features:
    * None for this iteration
  
* Scale into train, validate, test    
* Split into X_train, y_train, X_validate, y_validate, X_test, y_test


In [8]:
# Encode


In [9]:
# Split


In [10]:
# Assign


In [11]:
# Scale


<div class="alert alert-block alert-info"> 
    
## Models to Run:
    
**Classification:**
    
* Decision Tree
* K-Nearest Neighbors
* Random Forest
* Logistic Regression
 
---
    
#### Accuracy is the evaluation metric
    
#### Focus is to beat baseline and establish drivers of quality

In [14]:
# plot target


In [13]:
# do model stuff


<div class="alert alert-block alert-info">
    
## Modeling Conclusion:
    
*
*
*

<div class="alert alert-block alert-info">
    
# Project Conclusion
    

### Explore:
    
* 
  
* 
  
* 
  
* 
    
### Modeling:
    

    
### Recommendations and Next Steps:
 
* For the data engineers: 
* For the data scientsists: 
* For the business: 